In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import numpy.polynomial.polynomial as poly
import time
import hdbscan
import pandas as pd
from sklearn import linear_model
import matplotlib.animation as animation

In [ ]:
#function to do color based segmentation
def combined (img):
    #b, g, r = cv2.split(img)
    b = img[:, :, 0]
    g = img[:, :, 1]
    r = img[:, :, 2]
    #r_max = g_max = b_max = 255
    r_max = np.amax(r)
    g_max = np.amax(g)
    b_max = np.amax(b)
    #print (r)
    #cv2.imshow("red chaneel",b)

    red_norm = r/r_max
    green_norm = g/g_max
    blue_norm = b/b_max

   # print(red_norm.shape)

    norm = red_norm + blue_norm + green_norm


    small_num = 0.0001
    r = red_norm/(norm+small_num)
    g = green_norm/(norm+small_num)
    b = blue_norm/(norm+small_num)

    #print('normalized r g b values: %d %d %d' %(r, g, b))

    ExG = 2*g - r - b #excess green

    #ExGR = ExG -1.4*r - g #excess green minus red

    #CIVE = 0.441*r - 0.811*g + 0.385*b + 18.78745 #color index of vegetation extraction

    #redistribute the weights without VEG
    #w_ExG = 0.28
    #w_ExGR = 0.34
    #w_CIVE = 0.38

    #combined = w_ExG * ExG + w_ExGR * ExGR + w_CIVE * CIVE

    return ExG

In [ ]:
# to create animation
fig = plt.figure()
# ims = []

cap = cv2.VideoCapture(r'C:\Users\khan1\jupyter_test_code\2fps_crop_row.avi')

# fps = cap.get(cv2.CAP_PROP_FPS)
# print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
#Frames per second using video.get(cv2.CAP_PROP_FPS) : 60.0

fps = 2
title = 'normal speed video'
delay = int(1000 / fps)

#original video size
height = 1346  #height = frame.shape[0]
width = 1994   #width = frame.shape[1]

#resized video size for before cropping
new_height = 972 
new_width = 1444
dim = (new_width, new_height)


frame_counter = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    
    frame_counter += 1
    print("frame counter: ", frame_counter)
    
    #resized_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
#     height = frame.shape[0]
#     width = frame.shape[1]
#     print("Height:",height,"width:",width)

    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #crop_img = resized_frame[int(height / 5):int(height / 2 - 100), int(width / 3 - 100):int(300+(width / 3))]
    crop_img = frame[int(height / 2 - 300):int(height / 2 + 100), int(width / 2 - 300):int(300 + (width / 2))]
    
    img = crop_img
    
    cropheight = img.shape[0]
    cropwidth = img.shape[1]

    max_value = np.max(combined(img))
    min_value = np.min(combined(img))

    ###mapped combined image value (which is mostly negative) to 0-255
    new_min = 0
    new_max = 255
    old_range = max_value - min_value
    new_range = new_max - new_min
    lin_map = (((combined(img).astype(np.float64) - min_value) * new_range) / old_range) + new_min
    image_map = lin_map.astype(np.uint8)
    ###OTSU threshold
    thresh_val,thresh_img = cv2.threshold(image_map,0,255,cv2.THRESH_OTSU)

    #print(thresh_img)
    ###Morphology Opening-Closing to delete small weed segments
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh_img, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    
    ### all the white pixel coordinates in segemented image after noise clean
    indices = np.where(closing == [255])
    coordinates = list(zip(indices[0], indices[1]))
    
    ###convert list to array
    ###then flip the data from x-y format to y-x format for clsuter fit
    data = np.array(coordinates)
    y_x_data = np.flip(data)
    #print(y_x_data.shape)
    
    ##### cluster size for Veera video image ####
    cluster_size = 1000 
    
    
    ###Fit data to HDBscan
    ### min_cluster_size=100, has direct correlation with crop growth stage, type of crop
    ### NEED user input
    clusters = hdbscan.HDBSCAN(min_cluster_size=cluster_size).fit(y_x_data)
    labels = clusters.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    #print("clusters:", n_clusters)
    #good_points = clusters.exemplars_

    ###subtract outliers from original cluster data
    subtract_data = y_x_data
    
    """No iterative or only counting good points Steps are not added yet (from row_detect_all_v2)."""
    
    print("clusters:", n_clusters)
    #print(subtract_data.shape)

    ###convert list to array
    subtract_data = np.array(subtract_data)

    Ydata = subtract_data.T[0] #we plot this on x-axis
    Xdata = subtract_data.T[1] #we plot this on y_axis

    y_center = []
    x_center = []

    ### calculate center for each clusters ###
    for i in range(0,n_clusters):
    
        y_center.append(int(np.mean(np.array(Ydata[labels == i])))) #distance from (0,0) along x-axis
        x_center.append(int(np.mean(np.array(Xdata[labels == i])))) #distance from (0,0) along y-axis
    
    
#         plt.scatter(np.array(Ydata[labels == i]), np.array(Xdata[labels == i])) 
#         plt.plot(y_center[i],x_center[i], "kv",markersize=12)
#         plt.text(y_center[i] + 5,x_center[i], s = i+1, fontsize =10)
#         #print("cluster ",i+1,"center ", y_center[i])

#     plt.gca().invert_yaxis()
#     plt.show()
    
    # change this value according to  crop_roi size
    cluster_distance_threshold = 100  #this value can be updated based on user input or training data

    center_distance = 0

    ### calculate dynamic threshold
    center_sort = np.sort(y_center)


    percentage = 0.3 #bigger number means small distance constraint

    ###initialize with big numbers so that don't delete clusters (next cell) when don't meet following condition 
    keep = 10
    delete = 10

    ### currently the following loop works for only one outlier removal ####
    ### should apply it for multiple  outlier cluster removal ####

    deleted_cluster = []

    for i in range(0,n_clusters-1):
        for j in range(i+1,n_clusters):
        
            #print("i ",i,"j ", j)
        
            cluster_distance =  np.abs(y_center[i] - y_center[j])
            #print("cluster ",i+1,"and",j+1,"distance ",cluster_distance)
        
            if (cluster_distance < (cluster_distance_threshold - percentage * cluster_distance_threshold)):
                #print ("Cluster trouble",i+1,"and",j+1)
            
                ### Decide which cluster to keep based on shape and size ###
                ### Keep : Higher number of points, higher vertical distance ###
                ### Higher number of points may also indicate weeds, maybe just keep higher vertical height ###
            
                #get number of data points in cluster
                cluster_i_points = Xdata[labels == i].shape[0]
                cluster_j_points = Xdata[labels == j].shape[0]
            
                #get height of cluster
                cluster_i_height = np.abs(max(Xdata[labels == i]) - min(Xdata[labels == i]))
                cluster_j_height = np.abs(max(Xdata[labels == j]) - min(Xdata[labels == j]))
            
                if ((cluster_i_height >= cluster_j_height) & (cluster_i_points >= cluster_j_points)):
                
                    keep = i
                    delete = j
            
                else:
                
                    keep = j
                    delete = i
                
                #print("keep",keep+1)
                #print("delete",delete+1)
                deleted_cluster.append(delete)
                
#     for i in range(0,n_clusters):

#         if (any(i == item for item in deleted_cluster)):
#         #if (i == delete):
#             pass

#         else:
#             plt.scatter(np.array(Ydata[labels == i]), np.array(Xdata[labels == i]))
        
#     plt.gca().invert_yaxis()
#     plt.show()
    
    #### Fit line over clusters ####

    straight = 1
    curve = 2
    
    ransac = linear_model.RANSACRegressor()
    #### For Veera video image ###
    vertical_ROI_height = 400

    #print("number of clusters: ", n_clusters)
    #print("deleted clusters: ", len(deleted_cluster))
    
    total_points = 1000
    
    color = ['red', 'blue', 'cyan', 'magenta', 'yellow','black','green','red', 'blue', 'cyan']
    
    ## Only plot lines if number of correct cluser in ROI are 3 or 4
    if (2 < (n_clusters - len(deleted_cluster)) < 5):
        
        for i in range(0,n_clusters):

            if (any(i == item for item in deleted_cluster)):
                pass

            else:
                ransac.fit(Xdata[labels == i].reshape(-1,1), Ydata[labels == i].reshape(-1,1))
                xnew_cluster = np.linspace(0, vertical_ROI_height + 373, total_points)
                ffit_cluster = ransac.predict(xnew_cluster.reshape(-1,1))
                #plt.plot(ffit_cluster, xnew_cluster, '-',color = color[i])
                plt.scatter(ffit_cluster, xnew_cluster,color= color[i])
    else:
        pass
    
    
#     plt.gca().axes.get_xaxis().set_visible(False)
#     plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    #plt.savefig(r"C:\Users\khan1\jupyter_test_code\line_draw_img\{}.png".format(frame_counter),bbox_inches='tight',pad_inches = 0)
    plt.show()
    
"""Steps to create line drawn vidoe of original frame
1. save line draw image of cropped frame on C:\Users\khan1\jupyter_test_code\line_draw_img\{}.png"
2. open 'superimpose_crop_line_into_orig_frame.ipynb' and create image of line drawn on original frame
3. create video and saved in C:\Users\khan1\jupyter_test_code\detected_crop_row_orig_frame.avi"""
    
#     #cv2.imshow(title,frame)
# #     cv2.imshow("Cropped frame",img)
#     cv2.imshow("complete",img1)
#     if (cv2.waitKey(10) & 0xFF == ord('q')):
#         break
    
    if frame_counter > 65:
        break
    


print('Done!')

cap.release()
cv2.destroyAllWindows()